# Разработка синтаксического LL(1)-анализатора


**Цель**

Научиться разрабатывать LL(1)-анализатор формального языка, для которого сущеcтвует LL(1)-грамматика.

**Смысл**

С помощью формальных языков можно создавать пользовательские интерфейсы в ограниченных предметных областях, имитирующие человеческую речь.

**Задача**

Пусть дано слово в некотором алфавите. Определить с помощью LL(1)-анализатора
принадлежит ли данное слово формальному языку, заданному ниже, или нет:
* переменные,
* целочисленные константы,
* true, false,
* арифметические операции,
* логические операции,
* функции,
* предикаты

В соответствии с поставленным заданием была разработана LL(1)-грамматика:

END - конец слова	
1. <выражение> = 19 <множитель> 20 <список множителей> 21 <оператор>
2. <список множителей> = 22 * / 23 <множитель> 24 <список множителей>
3. <список множителей> = 25 END
4. <множитель> = 26 <слагаемое> 27 <список слагаемых>
5. <список слагаемых> = 28 + - 29 <слагаемое> 30 <список слагаемых>
6. <список слагаемых> = 31 END
7. <слагаемое> = 32 ( 33 <выражение> 34)
8. <слагаемое> = 35 const
9. <слагаемое> = 36 true false
10. <слагаемое> = 37 <переменная>
11. <слагаемое> = 38 <функция> 
12. <функция> = 39 function 40 ( 41 <переменная> 42 <список переменных>
13. <список переменных> = 43 ; 44 <переменная> 45 <список переменных>
14. <список переменных> = 46 ): 47 <тип>
15. <переменная> = 48 var: 49 <тип>
16. <тип> = 50 int bool char
17. <оператор> = 51 && || 52 <выражение>
18. <оператор> = 53 END






In [110]:
from collections import deque
import pandas as pd
table = {}

In [111]:
data = pd.read_excel("table.xlsx", index_col = "№")
data.head()

,множество направляющих символов,next,return,stack,accept,error
№,,,,,,
1,end ) ( const var: true false function && ||,19,0,0,0,1
2,* /,22,0,0,0,0
3,end ) && ||,25,0,0,0,1
4,( const var: true false function,26,0,0,0,1
5,+ -,28,0,0,0,0


In [112]:
for i in range(1, data.shape[0]+1):
  table[i] = []
  for j in data.columns:
    table[i].append(data[j][i])

In [113]:
table

{1: ['end ) ( const var: true false function && ||', 19, 0, 0, 0, 1],
 2: ['* / ', 22, 0, 0, 0, 0],
 3: ['end ) && ||', 25, 0, 0, 0, 1],
 4: ['( const var: true false function', 26, 0, 0, 0, 1],
 5: ['+ -', 28, 0, 0, 0, 0],
 6: ['* / ) && || end', 31, 0, 0, 0, 1],
 7: ['(', 32, 0, 0, 0, 0],
 8: ['const', 35, 0, 0, 0, 0],
 9: ['true false', 36, 0, 0, 0, 0],
 10: ['var:', 37, 0, 0, 0, 0],
 11: ['function', 38, 0, 0, 0, 1],
 12: ['function', 39, 0, 0, 0, 1],
 13: [';', 43, 0, 0, 0, 0],
 14: ['):', 46, 0, 0, 0, 1],
 15: ['var:', 48, 0, 0, 0, 1],
 16: ['int bool char', 50, 0, 0, 0, 1],
 17: ['&& || ', 51, 0, 0, 0, 0],
 18: ['&& || * / + - ) end', 53, 0, 0, 0, 1],
 19: ['( const var: true false function', 4, 0, 1, 0, 1],
 20: ['* / ) end', 2, 0, 1, 0, 1],
 21: ['&& || * / + - ) end', 17, 0, 0, 0, 1],
 22: ['* /', 23, 0, 0, 1, 1],
 23: ['( const var: true false function', 4, 0, 1, 0, 1],
 24: ['* / ) && || end', 2, 0, 0, 0, 1],
 25: ['end ) && ||', -1, 1, 0, 0, 1],
 26: ['( const var: true fa

In [114]:
def analyzer(word):
  n = 1
  word = word.split()
  word.append("end")
  stack = deque()

  for i in range(len(word)):
    
    symbol = word[i]
    print("Символ === ", symbol)

    is_accept = False
    is_error = False
    is_end_symbol = False

    while not(is_accept) or not(is_error) or not(is_end_symbol) :
      print(n, " ---> ")

      is_end_symbol = (symbol == "end" and (symbol in table[n][0]) and len(stack) == 0)

      symbol = ("const" if symbol.isdigit() else symbol)

      m = "; ): && || ) ( true false * / + - int bool char end const function".split()
      if i != len(word)-1 and (symbol not in m):
        symbol = ("function" if ((word[i+1] == '(')) else symbol)
      
      
      symbol = ("var:" if symbol not in m else symbol)
      
      if ((symbol in table[n][0]) and not(is_end_symbol)):

        # Return
        new_n = n
        if table[n][2] ==  1:
          print(n, "Берем из стека")
          new_n = stack.pop()
          print(stack)
        else:
          new_n = table[n][1]

        # Statck
        if table[n][3] ==  1:
          print(n, "Записываем в стек")
          stack.append(n+1)
          print(stack)

        # Accept
        if table[n][4] ==  1:
          is_accept = True
          print(n, ".:Accept:.")
          n = new_n
          break

        n = new_n

      elif is_end_symbol:
        break
      # Error
      elif table[n][5] ==  0:
          n += 1
      else: 
        is_error = True;
        break
    
    if is_error: 
      print(".!Error!.")
      break

    if is_end_symbol:
      print("================================") 
      print("Слово принадлежит данному языку.")
      break

При вводе слова для проверки все символы необходимо разделять пробелом.

In [115]:
analyzer("5 * 100 && 234 ")

Символ ===  5
1  ---> 
19  ---> 
19 Записываем в стек
deque([20])
4  ---> 
26  ---> 
26 Записываем в стек
deque([20, 27])
7  ---> 
8  ---> 
35  ---> 
35 Берем из стека
deque([20])
35 .:Accept:.
Символ ===  *
27  ---> 
5  ---> 
6  ---> 
31  ---> 
31 Берем из стека
deque([])
20  ---> 
20 Записываем в стек
deque([21])
2  ---> 
22  ---> 
22 .:Accept:.
Символ ===  100
23  ---> 
23 Записываем в стек
deque([21, 24])
4  ---> 
26  ---> 
26 Записываем в стек
deque([21, 24, 27])
7  ---> 
8  ---> 
35  ---> 
35 Берем из стека
deque([21, 24])
35 .:Accept:.
Символ ===  &&
27  ---> 
5  ---> 
6  ---> 
31  ---> 
31 Берем из стека
deque([21])
24  ---> 
2  ---> 
3  ---> 
25  ---> 
25 Берем из стека
deque([])
21  ---> 
17  ---> 
51  ---> 
51 .:Accept:.
Символ ===  234
52  ---> 
1  ---> 
19  ---> 
19 Записываем в стек
deque([20])
4  ---> 
26  ---> 
26 Записываем в стек
deque([20, 27])
7  ---> 
8  ---> 
35  ---> 
35 Берем из стека
deque([20])
35 .:Accept:.
Символ ===  end
27  ---> 
5  ---> 
6  ---> 
31  --->

In [116]:
analyzer("( 534 + 34 * ( 43 + 5 + 3 * 6 ) )")

Символ ===  (
1  ---> 
19  ---> 
19 Записываем в стек
deque([20])
4  ---> 
26  ---> 
26 Записываем в стек
deque([20, 27])
7  ---> 
32  ---> 
32 .:Accept:.
Символ ===  534
33  ---> 
33 Записываем в стек
deque([20, 27, 34])
1  ---> 
19  ---> 
19 Записываем в стек
deque([20, 27, 34, 20])
4  ---> 
26  ---> 
26 Записываем в стек
deque([20, 27, 34, 20, 27])
7  ---> 
8  ---> 
35  ---> 
35 Берем из стека
deque([20, 27, 34, 20])
35 .:Accept:.
Символ ===  +
27  ---> 
5  ---> 
28  ---> 
28 .:Accept:.
Символ ===  34
29  ---> 
29 Записываем в стек
deque([20, 27, 34, 20, 30])
7  ---> 
8  ---> 
35  ---> 
35 Берем из стека
deque([20, 27, 34, 20])
35 .:Accept:.
Символ ===  *
30  ---> 
5  ---> 
6  ---> 
31  ---> 
31 Берем из стека
deque([20, 27, 34])
20  ---> 
20 Записываем в стек
deque([20, 27, 34, 21])
2  ---> 
22  ---> 
22 .:Accept:.
Символ ===  (
23  ---> 
23 Записываем в стек
deque([20, 27, 34, 21, 24])
4  ---> 
26  ---> 
26 Записываем в стек
deque([20, 27, 34, 21, 24, 27])
7  ---> 
32  ---> 
32 .:

In [117]:
analyzer("var: bool")

Символ ===  var:
1  ---> 
19  ---> 
19 Записываем в стек
deque([20])
4  ---> 
26  ---> 
26 Записываем в стек
deque([20, 27])
7  ---> 
8  ---> 
9  ---> 
10  ---> 
37  ---> 
15  ---> 
48  ---> 
48 .:Accept:.
Символ ===  bool
49  ---> 
16  ---> 
50  ---> 
50 Берем из стека
deque([20])
50 .:Accept:.
Символ ===  end
27  ---> 
5  ---> 
6  ---> 
31  ---> 
31 Берем из стека
deque([])
20  ---> 
Слово принадлежит данному языку.


In [118]:
analyzer("function ( var: int ): int")

Символ ===  function
1  ---> 
19  ---> 
19 Записываем в стек
deque([20])
4  ---> 
26  ---> 
26 Записываем в стек
deque([20, 27])
7  ---> 
8  ---> 
9  ---> 
10  ---> 
11  ---> 
38  ---> 
12  ---> 
39  ---> 
39 .:Accept:.
Символ ===  (
40  ---> 
40 .:Accept:.
Символ ===  var:
41  ---> 
41 Записываем в стек
deque([20, 27, 42])
15  ---> 
48  ---> 
48 .:Accept:.
Символ ===  int
49  ---> 
16  ---> 
50  ---> 
50 Берем из стека
deque([20, 27])
50 .:Accept:.
Символ ===  ):
42  ---> 
13  ---> 
14  ---> 
46  ---> 
46 .:Accept:.
Символ ===  int
47  ---> 
16  ---> 
50  ---> 
50 Берем из стека
deque([20])
50 .:Accept:.
Символ ===  end
27  ---> 
5  ---> 
6  ---> 
31  ---> 
31 Берем из стека
deque([])
20  ---> 
Слово принадлежит данному языку.


In [119]:
analyzer("true * true && var: char + function ( var: int ): char")

Символ ===  true
1  ---> 
19  ---> 
19 Записываем в стек
deque([20])
4  ---> 
26  ---> 
26 Записываем в стек
deque([20, 27])
7  ---> 
8  ---> 
9  ---> 
36  ---> 
36 Берем из стека
deque([20])
36 .:Accept:.
Символ ===  *
27  ---> 
5  ---> 
6  ---> 
31  ---> 
31 Берем из стека
deque([])
20  ---> 
20 Записываем в стек
deque([21])
2  ---> 
22  ---> 
22 .:Accept:.
Символ ===  true
23  ---> 
23 Записываем в стек
deque([21, 24])
4  ---> 
26  ---> 
26 Записываем в стек
deque([21, 24, 27])
7  ---> 
8  ---> 
9  ---> 
36  ---> 
36 Берем из стека
deque([21, 24])
36 .:Accept:.
Символ ===  &&
27  ---> 
5  ---> 
6  ---> 
31  ---> 
31 Берем из стека
deque([21])
24  ---> 
2  ---> 
3  ---> 
25  ---> 
25 Берем из стека
deque([])
21  ---> 
17  ---> 
51  ---> 
51 .:Accept:.
Символ ===  var:
52  ---> 
1  ---> 
19  ---> 
19 Записываем в стек
deque([20])
4  ---> 
26  ---> 
26 Записываем в стек
deque([20, 27])
7  ---> 
8  ---> 
9  ---> 
10  ---> 
37  ---> 
15  ---> 
48  ---> 
48 .:Accept:.
Символ ===  char
49 